<a href="https://colab.research.google.com/github/syh0397/Statistics_python/blob/main/13_%EC%9A%94%EC%9D%B8%EB%B6%84%EC%84%9D(FA)(Factor_Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pingouin
!pip install --upgrade pingouin
!pip install scikit_posthocs
!pip install --upgrade patsy
!pip install patsy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd 
import numpy as np 

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import sklearn 
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)

import scipy 
from scipy import  stats
import statsmodels.api as sm
import pylab as py

import pingouin as pg
import scikit_posthocs

pd.options.display.float_format = '{:.3f}'.format

## 요인분석(FA)(Factor Analysis)
-  변수 간 상관관계를 바탕으로 **다변량 자료를 축소 또는 요약**
   - 사전지식이나 이론적 배경의 유무에 따라서 **탐색적 방법**과 **확인적 방법으로 구분**
   - 다변량 = 변수 많음
   - 여러개 변수를 어떻게 공통요인으로 묶어서 차원축소 할 것인가? 에 대한 대답중 하나 
---

- 이전에 다뤘던것과는 다르게 독립변수 X 혹은 종속변수 Y같은 개념이 없다.
- 변수 특성을 파악할때도 사용할 수 있다.
  - 요인들의 상호 독립적인 특성을 파악할 수 있다. 
  - 서로 상관관계가 높은 변수들끼리 그룹핑하는 기법이므로 당연히 상관계수가 높을수록 요인분석에 유리하다.
  - 일반적으로 상관계수 ±3 이하의 자료는 요인분석에 부적합 하다고 한다.


In [ ]:
!pip install factor_analyzer

In [11]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'geyser',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'taxis',
 'tips',
 'titanic']

In [19]:
from factor_analyzer import FactorAnalyzer
import numpy as np
from sklearn.preprocessing import LabelEncoder


hos = sns.load_dataset('titanic')
hos

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.000,1,0,7.250,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.000,1,0,71.283,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.000,0,0,7.925,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.000,1,0,53.100,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.000,0,0,8.050,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000,0,0,13.000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.000,0,0,30.000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.450,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.000,0,0,30.000,C,First,man,True,C,Cherbourg,yes,True


In [21]:
hos["sex"] = LabelEncoder().fit_transform(hos.sex)
hos["embarked"] = LabelEncoder().fit_transform(hos.embarked)
hos["who"] = LabelEncoder().fit_transform(hos.who)
hos["adult_male"] = LabelEncoder().fit_transform(hos.adult_male)
hos["deck"] = LabelEncoder().fit_transform(hos.deck)
hos["embark_town"] = LabelEncoder().fit_transform(hos.embark_town)
hos["alone"] = LabelEncoder().fit_transform(hos.alone)
hos["alive"] = LabelEncoder().fit_transform(hos.alive)

In [22]:
hos

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,1,22.000,1,0,7.250,2,Third,1,1,7,2,0,0
1,1,1,0,38.000,1,0,71.283,0,First,2,0,2,0,1,0
2,1,3,0,26.000,0,0,7.925,2,Third,2,0,7,2,1,1
3,1,1,0,35.000,1,0,53.100,2,First,2,0,2,2,1,0
4,0,3,1,35.000,0,0,8.050,2,Third,1,1,7,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000,0,0,13.000,2,Second,1,1,7,2,0,1
887,1,1,0,19.000,0,0,30.000,2,First,2,0,1,2,1,1
888,0,3,0,NaN,1,2,23.450,2,Third,2,0,7,2,0,0
889,1,1,1,26.000,0,0,30.000,0,First,1,1,2,0,1,1


In [23]:
columns = hos.columns
columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [ ]:
# non_tans = ['pickup', 'dropoff', 'passengers', 'distance', 'fare', 'tip', 'tolls',
#        'total']
# trans = ['color', 'payment', 'pickup_zone', 'dropoff_zone',
#        'pickup_borough', 'dropoff_borough']

In [24]:
df = hos[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked',  'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone']]
# 5개의 요인으로 축소 
fa = FactorAnalyzer(method = 'principal',
                    n_factors = 5,
                    rotation = 'varimax').fit(df)

In [26]:
pd.DataFrame(fa.loadings_, index = df.columns)

,0,1,2,3,4
survived,-0.919,0.016,-0.249,-0.063,0.111
pclass,0.162,-0.044,0.887,0.039,-0.078
sex,0.589,0.295,-0.044,0.071,-0.682
age,0.390,0.323,-0.466,0.049,0.491
sibsp,0.057,-0.803,0.033,0.050,-0.082
parch,-0.051,-0.784,-0.036,0.041,0.038
fare,-0.113,-0.296,-0.707,-0.166,0.033
embarked,0.066,-0.021,0.114,0.989,-0.020
who,-0.204,0.103,-0.117,-0.016,0.893
adult_male,0.650,0.442,-0.084,0.062,-0.487


- 나온 값들은 요인 적재값을 나타낸다. (loadings_ => factorloadings_

  - 요인 적재값이란 ?
    - 변수와 요인의 관련성 (위에서는 세로축이 변수(행)) 정도를 나타내는 것
    - 위에서는 요인이 0인경우가 survived와 가장 높은 연관성을 가지고 있다고 볼 수 있다. 

In [28]:
pd.DataFrame(fa.get_communalities(), index = df.columns)

,0
survived,0.923
pclass,0.822
sex,0.906
age,0.718
sibsp,0.658
parch,0.622
fare,0.629
embarked,0.997
who,0.863
adult_male,0.866


- 하나의 요인이 변수를 얼마나 설명할수 있나 ? 하는 부분 (공통성)

- 공통성(communality)이란?
  - 각 변수의 변량 중 분석에 포함된 요인들에 의해 설명되는 공통분산의 비율로, 변수 별로 요인적재량의 제곱을 더해 구한다
- PCA를 설명하는것과 닮았다 
  - 그럼 고윳값을 볼수 있나 ? (아이겐 밸류)
    - 물론 볼 수 있다.


In [30]:
ev, v = fa.get_eigenvalues()
pd.DataFrame(ev)

,0
0,4.145
1,2.496
2,1.923
3,1.727
4,1.139
5,0.650
6,0.576
7,0.475
8,0.320
9,0.270


- 요인 0의 경우에는 4.145개의 측정변수를 설명할 수 있다.
- 요인 5부터는 1 미만의 설명이 나오므로 ,,,, 5가 아니라 4로 측정했어야 한다고 볼 수 있다.


In [31]:
fa.transform(df.dropna())

array([[ 2.47313153, -0.57694087,  2.09608307,  1.42736247, -1.81077042],
       [-4.9431894 , -1.65295093, -5.12185472, -4.57881304,  3.56548526],
       [-3.84339817,  0.66570701,  1.27443953,  0.94090625,  2.74876962],
       ...,
       [-4.90937266,  0.43895912, -3.40525186,  0.49386053,  2.75704797],
       [-2.0573371 ,  2.07329734, -3.88062154, -4.12092662, -0.81370889],
       [ 2.85444935,  2.15173984,  1.73456781, -0.96667722, -1.35369589]])

- 요인 적재값이 변수와 요인의 상관정도를 ( 상관관계) 나타내는데 
- 요인 점수의 경우에는 개체와 요인의 상관 정도를 나타낸다.

---

그럼 이걸로 무엇을 할 수 있는가 ? 

 - 측정도구의 타당성을 평가하는 데 요인분석을 사용할 수 있다. 연구자가 어떤 개념을 여러
가지 변수를 사용하여 측정했을 때 요인분석 결과에서 하나의 요인으로 묶이는 변수들은
측정의 타당성이 있다고 볼 수 있으며 그렇지 못한 변수들은 상이한 개념을 측정하는
변수로 간주하고 다음 조사에서 제거할 수 있다

- 회귀분석이나 판별분석 등 추가적인 분석을 실시할 때, 수많은 변수에 의해서 다중공선성과
같은 문제로 분석기법사용에 어려움이 많을 수 있다. 이러한 추가적인 분석을
용이하게 하기 위해 여러 개의 변수를 소수의 새로운 요인으로 축소하고 각각의
요인이 독립적(orthogonal)이 되게 하기 위해 요인분석을 이용할 수 있다.

- 